In [3]:
import sys
import numpy as np
import pandas as pd 
import csv

In [4]:
f = open('warfrin.csv','rU')
csvr = csv.reader(f, dialect=csv.excel)
header = next(csvr)
data   = list(csvr)
f.close()

/Users/xiaojicmao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 'U' mode is deprecated
  """Entry point for launching an IPython kernel.


In [5]:
# Only keep subjects that reached stable dose of warfarin and stable observed INR
# [37]: 'Subject Reached Stable Dose of Warfarin'
# [38]: 'Therapeutic Dose of Warfarin'
data = [x for x in data if x[38]!='NA' and x[38] != "" and x[37]=='1']
# the levels of ages increasing order:
agegroups = sorted(set(x[8].replace('NA','') for x in data))

In [6]:
xmap = lambda x: \
[   # recommended dose: change data type to float  
    ("therapeut_dose",  float(x[38])),
    # gender 0: one-hot-encoding 
    ('Male?',x[3]=='male'),
    ('Female?', x[3] == 'female'),
    # race 1:3: one-hot-encoding 
    ('White.',x[5]=='White'),
    ('Asian.',x[5]=='Asian'),
    ('Black.',x[5]=='Black or African American'),
    # age 4:5: multiple-valued encoding for age 
    ('Age group',agegroups.index(x[8]) if x[8] in agegroups else 0), # this exactly gives the decades of age 
    ('No age?',x[8] not in agegroups or agegroups.index(x[8]) == 0),
    # height 6, weight 7: change data type to float 
    ('Height',float(x[9]) if x[9] not in ('NA', '') else 0.),
    ('Weight',float(x[10]) if x[10] not in ('NA', '') else 0.),
] + [# Indication for Warfarin Treatment 8:15: one-hot encoding 
    ('Indication for Warfarin Treatment: '+str(i), str(i) in x[11])
    for i in range(1,9)
] + [# diabetes 16:17: one-hot-encoding 
    ('Diabetes=0?',x[13]=='0'),
    ('Diabetes=1?',x[13]=='1'),
] + [#Congestive Heart Failure and/or Cardiomyopathy 18:19: one-hot-encoding 
    ('Congestive Heart Failure and/or Cardiomyopathy=0?',x[14]=='0'),
    ('Congestive Heart Failure and/or Cardiomyopathy=1?',x[14]=='1'),
] + [#Valve Replacement 20:21: one-hot-encoding 
    ('Valve Replacement=0?',x[15]=='0'),
    ('Valve Replacement=1?',x[15]=='1'),
] + [
    # Aspirin 22:23: one-hot-encoding 
    ('aspirin=0?',x[17]=='0'),
    ('aspirin=1?',x[17]=='1'),
    # Acetaminophen or Paracetamol (Tylenol) 24:25: one-hot-encoding 
    ('Acetaminophen=0?',x[18]=='0'),
    ('Acetaminophen=1?',x[18]=='1'),
    # Was Dose of Acetaminophen or Paracetamol (Tylenol) >1300mg/day 26:27: one-hot-encoding 
    ('Acetaminophen hi dose=0?',x[19]=='0'),
    ('Acetaminophen hi dose=1?',x[19]=='1'),
    # Simvastatin (Zocor) 28:29: one-hot-encoding 
    ('Simvastatin=0?',x[20]=='0'),
    ('Simvastatin=1?',x[20]=='1'),
    # Atorvastatin (Lipitor) 30:31: one-hot-encoding 
    ('Atorvastatin=0?',x[21]=='0'),
    ('Atorvastatin=1?',x[21]=='1'),
    # Fluvastatin (Lescol) 32:33: one-hot-encoding 
    ('Fluvastatin=0?',x[22]=='0'),
    ('Fluvastatin=1?',x[22]=='1'),
    # Lovastatin (Mevacor) 34:35 : one-hot-encoding 
    ('Lovastatin=0?',x[23]=='0'),
    ('Lovastatin=1?',x[23]=='1'),
    # Pravastatin (Pravachol) 36:37: one-hot-encoding 
    ('Pravastatin=0?',x[24]=='0'),
    ('Pravastatin=1?',x[24]=='1'),
    # Rosuvastatin (Crestor) 38:39: one-hot-encoding 
    ('Rosuvastatin=0?',x[25]=='0'),
    ('Rosuvastatin=1?',x[25]=='1'),
    # Cerivastatin (Baycol) 40:41: one-hot-encoding 
    ('Cerivastatin=0?',x[26]=='0'),
    ('Cerivastatin=1?',x[26]=='1'),
    # Amiodarone (Cordarone) 42:43: one-hot-encoding 
    ('Amiodarone=0?',x[27]=='0'),
    ('Amiodarone=1?',x[27]=='1'),
    # Carbamazepine (Tegretol) 44:45: one-hot-encoding  
    ('Carbamazepine=0?',x[28]=='0'),
    ('Carbamazepine=1?',x[28]=='1'),
    # Phenytoin (Dilantin): one-hot-encoding 
    ('Phenytoin=0?',x[29]=='0'),
    ('Phenytoin=1?',x[29]=='1'),
    # Rifampin or Rifampicin: one-hot-encoding 
    ('Rifampin=0?',x[30]=='0'),
    ('Rifampin=1?',x[30]=='1'),
    # Sulfonamide Antibiotics: one-hot-encoding 
    ('Sulfonamide Antibiotics=0?',x[31]=='0'),
    ('Sulfonamide Antibiotics=1?',x[31]=='1'),
    # Macrolide Antibiotics: one-hot-encoding 
    ('Macrolide Antibiotics=0?',x[32]=='0'),
    ('Macrolide Antibiotics=1?',x[32]=='1'),
    # Anti-fungal Azoles: one-hot-encoding 
    ('Anti-fungal Azoles=0?',x[33]=='0'),
    ('Anti-fungal Azoles=1?',x[33]=='1'),
    # Herbal Medications, Vitamins, Supplements: one-hot-encoding 
    ('Herbal Medications, Vitamins, Supplements=0?',x[34]=='0'),
    ('Herbal Medications, Vitamins, Supplements=1?',x[34]=='1'),
] + [
    #smoker: one-hot-encoding 
    ('Smoker=0?',x[40]=='0'),
    ('Smoker=1?',x[40]=='1'),
] + [
    # CYP2C9 consensus: all other values are grouped into a single level, one-hot-encoding 
    ('CYP2C9 *1/*1',x[59]=='*1/*1'),
    ('CYP2C9 *1/*2',x[59]=='*1/*2'),
    ('CYP2C9 *1/*3',x[59]=='*1/*3'),
    ('CYP2C9 NA',x[59]=='' or x[59]=='NA'),
    # VKORC1 -1639 consensus: one-hot-encoding 
    ('VKORC1 -1639 A/A',x[60]=='A/A'),
    ('VKORC1 -1639 A/G',x[60]=='A/G'),
    ('VKORC1 -1639 G/G',x[60]=='G/G'),
    # VKORC1 497 consensus: one-hot-encoding 
    ('VKORC1 497 T/T',x[61]=='T/T'),
    ('VKORC1 497 G/T',x[61]=='G/T'),
    ('VKORC1 497 G/G',x[61]=='G/G'),
    # VKORC1 1173 consensus: one-hot-encoding 
    ('VKORC1 1173 T/T',x[62]=='T/T'),
    ('VKORC1 1173 C/T',x[62]=='C/T'),
    ('VKORC1 1173 C/C',x[62]=='C/C'),
    # VKORC1 1542 consensus: one-hot-encoding 
    ('VKORC1 1542 C/C',x[63]=='C/C'),
    ('VKORC1 1542 C/G',x[63]=='C/G'),
    ('VKORC1 1542 G/G',x[63]=='G/G'),
    # VKORC1 3730 consensus: one-hot-encoding 
    ('VKORC1 3730 A/A',x[64]=='A/A'),
    ('VKORC1 3730 A/G',x[64]=='A/G'),
    ('VKORC1 3730 G/G',x[64]=='G/G'),
    # VKORC1 2255 consensus: one-hot-encoding 
    ('VKORC1 2255 C/C',x[65]=='C/C'),
    ('VKORC1 2255 C/T',x[65]=='C/T'),
    ('VKORC1 2255 T/T',x[65]=='T/T'),
    # VKORC1 -4451 consensus: one-hot-encoding 
    ('VKORC1 -4451 C/C',x[66]=='C/C'),
    ('VKORC1 -4451 A/C',x[66]=='A/C'),
    ('VKORC1 -4451 A/A',x[66]=='A/A')
]

In [7]:
X       = np.array([list(zip(*xmap(x)))[1] for x in data])
Xnames  = np.array(list(zip(*xmap(data[0])))[0])
name_list = list(Xnames)

In [11]:
# remove obs with unknown race (461 obs with value "")
goodidx = X[:, name_list.index('White.')] + X[:, name_list.index('Asian.')] + X[:, name_list.index('Black.')] == 1
X = X[goodidx, :]
# remove obs with unknown gender (the same 461 obs as race (""), plus 4 extra obs ("NA"))
goodidx = X[:, name_list.index('Male?')] + X[:, name_list.index('Female?')] == 1
X = X[goodidx, :]
# remove obs with missing age: exactly the same 461 obs have un knwon race (""), plus 42 extra obs ("NA")
goodidx = X[:, name_list.index('No age?')] != 1
X = X[goodidx, :]

In [12]:
final_data = pd.DataFrame(data = X, columns = Xnames)

In [13]:
final_data.to_csv("warfarin_clean.csv")